# MAXIMAL COVERAGE ALGORITHM

## Libraries

In [156]:
import numpy as np
import os
import pandas as pd
import re

## Constants

In [90]:
data_path = "/Users/Jovan/Desktop/MDA-Project/Data"

input_path_segmented = "/2_DataSegmented"
input_path_dist = "/6_DistanceMatricesTest"
temp_path = "/Users/Jovan/Desktop"

cities = ["Antwerpen", "Brugge", "Brussels", "Charleroi", "Gent", "Leuven", "Liege", "Oostende"]
cities = ["Oostende"]

COVERAGE_RADIUS = 150

## Functions

In [92]:
# Function which removes .1 .2 .3 ... after )

def extract_coordinates(s):
    s = s.split('(')[1].split(')')[0].split(', ')
    s = map(float, s)
    return tuple(s)

## Importing data

In [80]:
os.chdir(data_path + input_path_dist)
city_data = {}

for city in cities:
        city_data[city] = pd.read_csv(city + "_cost_matrix.csv", index_col=0)
        print(f"Data for {city} loaded successfully.")

Data for Oostende loaded successfully.


## Comparisons

In [81]:
for city, df in city_data.items():
    df = df.apply(pd.to_numeric, errors='coerce')
    
    num_columns = df.shape[1]
    mid_index = num_columns // 2

    old_aed_columns = df.iloc[:, :mid_index]
    new_aed_columns = df.iloc[:, mid_index:]

    df['Old_AED_within_150m'] = old_aed_columns.apply(lambda row: (row < 150).any(), axis=1)
    df['New_AED_within_150m'] = new_aed_columns.apply(lambda row: (row < 150).any(), axis=1)

    percentage_old_aed = df['Old_AED_within_150m'].mean() * 100
    percentage_new_aed = df['New_AED_within_150m'].mean() * 100

    print(f"Results for {city}:")
    print(f"Percentage of cardiac arrests with an AED within 150m (old placements): {percentage_old_aed:.2f}%")
    print(f"Percentage of cardiac arrests with an AED within 150m (new placements): {percentage_new_aed:.2f}%\n")


Results for Oostende:
Percentage of cardiac arrests with an AED within 150m (old placements): 18.00%
Percentage of cardiac arrests with an AED within 150m (new placements): 32.00%



In [24]:
# # Same AEDs are being picked by the algorithm more than once

# for city, df in city_data.items():
#     num_columns = df.shape[1]
#     mid_index = num_columns // 2
#     num_unique_columns = df.iloc[:, mid_index:].columns.nunique()
    
#     print(f"City: {city} Number of unique column labels:", num_unique_columns, mid_index)

City: Antwerpen Number of unique column labels: 638 722
City: Brugge Number of unique column labels: 152 161
City: Brussels Number of unique column labels: 1674 2024
City: Charleroi Number of unique column labels: 323 365
City: Gent Number of unique column labels: 362 389
City: Leuven Number of unique column labels: 153 167
City: Liege Number of unique column labels: 631 686
City: Oostende Number of unique column labels: 62 69


## Creating old_aeds, new_aeds and nearest aed dataframes

In [163]:
for city in cities:
    df = city_data[city]

    num_columns = df.shape[1]
    mid_index = num_columns // 2

    # First half of columns are old aeds, second half are new aeds
    old_aed_columns = df.columns[:mid_index]
    new_aed_columns = df.columns[mid_index:]

    old_aeds_df = pd.DataFrame(old_aed_columns, columns=["Old AEDs"])
    new_aeds_df = pd.DataFrame(new_aed_columns, columns=["New AEDs"])
    # print(old_aeds_df)
    # print(new_aeds_df)

    old_aeds_df[['lat', 'long']] = old_aeds_df['Old AEDs'].apply(extract_coordinates).apply(pd.Series)
    old_aeds_df.drop(columns=['Old AEDs'], inplace=True)
    new_aeds_df[['lat', 'long']] = new_aeds_df['New AEDs'].apply(extract_coordinates).apply(pd.Series)
    new_aeds_df.drop(columns=['New AEDs'], inplace=True)
    
    # print(old_aeds_df)
    # print(new_aeds_df)
    # Export it wherever you want, I'll clean it up later
    # os.chdir(temp_path)
    # export ...

    os.chdir(data_path + input_path_segmented)
    cards = pd.read_csv(city + "_cards_test.csv")[['latitude', 'longitude']]
    cards.rename(columns={'latitude': 'card_lat'}, inplace=True)
    cards.rename(columns={'longitude': 'card_long'}, inplace=True)
    df_new = df.iloc[:, mid_index:]
    min_indices = np.argmin(df_new, axis=1)

    nearest_aed = pd.DataFrame({
    'card_lat': cards['card_lat'].values,
    'card_long': cards['card_long'].values,
    'aed_lat': new_aeds_df.iloc[min_indices]['lat'].values,
    'aed_long': new_aeds_df.iloc[min_indices]['long'].values
    })
    nearest_aed['distance'] = None
    for i, idx in enumerate(min_indices):
        nearest_aed.at[i, 'distance'] = df_new.iat[i, idx]

    print(nearest_aed)
    # Export it wherever you want, I'll clean it up later
    # os.chdir(temp_path)
    # export ...

     card_lat  card_long    aed_lat  aed_long distance
0   51.232792   2.944167  51.234931  2.941995    287.0
1   51.228196   2.910508  51.228294  2.910614     14.0
2   51.229511   2.913998  51.230139  2.915366    118.0
3   51.234277   2.915419  51.232372  2.915172    276.0
4   51.234876   2.916587  51.234169  2.918209    177.0
..        ...        ...        ...       ...      ...
95  51.231357   2.945571  51.234931  2.941995    491.0
96  51.211095   2.878669  51.210841  2.876398    226.0
97  51.230009   2.915204  51.230139  2.915366     17.0
98  51.219038   2.909445  51.219186  2.907753    222.0
99  51.216481   2.890583  51.215484  2.888037    235.0

[100 rows x 5 columns]
